In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, Conv2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf 

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/drive/My Drive/iste_project/data1/CASIA-WebFace/")
!ls


0000045  0000554  0001116  0001633  0003633  0005015  0005420  0032658	0085860
0000099  0000555  0001117  0001634  0003697  0005016  0005424  0032741	0086205
0000100  0000562  0001118  0001638  0003777  0005020  0005428  0032962	0086703
0000102  0000563  0001126  0001640  0003779  0005022  0005429  0033422	0086883
0000103  0000568  0001127  0001641  0003807  0005023  0005430  0033509	0087104
0000105  0000570  0001129  0001642  0003888  0005024  0005431  0034305	0087109
0000107  0000571  0001131  0001650  0003928  0005025  0005435  0034309	0087726
0000108  0000574  0001138  0001652  0003931  0005026  0005436  0034365	0087799
0000114  0000575  0001139  0001653  0003941  0005029  0005437  0034390	0088023
0000117  0000579  0001143  0001661  0003981  0005036  0005438  0034492	0088144
0000119  0000580  0001146  0001662  0004051  0005038  0005439  0034733	0088298
0000121  0000582  0001147  0001664  0004056  0005041  0005441  0035060	0088502
0000133  0000583  0001151  0001667  0004081  0005042

In [0]:
path = "/content/drive/My Drive/iste_project/data1/CASIA-WebFace/"
files=os.listdir(path)
print(len(files))




1799


In [0]:
main = []
temp =[]
for i in range(1,100):
  path=  str(files[i])
  print(path)
  folder=os.listdir(path)
  pics=len(folder)
  #print(pics)
  for j in range(1,pics):
    path1 = path + "/"+ str(folder[j]) 
    img = cv2.imread(path1,0)
    temp.append(img)
  temp = np.reshape(temp,(1,pics,250,250))
  main.append(temp)

0005534


ValueError: ignored

In [0]:
main = np.array(main)
print(main.shape)
#main = np.reshape(main,(100,147,250,250))
print(main.shape)


(221, 221, 250, 250)
(221, 221, 250, 250)


In [0]:
for n in range(5):
  i,j = np.random.randint(0,100), np.random.randint(0,pics)
  print("Candidate number",i,",image number",j)
  plt.imshow(main[i,j], cmap = 'gray')
  plt.show()

Candidate number 9 ,image number 18


IndexError: ignored

In [0]:
triplets = []
for i in range(1024):
  x = np.random.randint(0,40)
  y = np.random.randint(0,40)
  while(x == y):
    y = np.random.randint(0,40)
  anchor = np.random.randint(0,10)
  positive = np.random.randint(0,10)
  while(anchor == positive):
    positive= np.random.randint(0,10)
  negative = np.random.randint(0,10)
  triplets.append([[x,anchor],[x,positive],[y,negative]])
 
triplets = np.array(triplets)
print(triplets.shape)

(1024, 3, 2)


In [0]:
for i in range(5):
  p = np.random.randint(0,1024)
  print("FP number ", p,":")
  a = main[triplets[p,0,0],triplets[p][0][1]]
  b = main[triplets[p][1][0],triplets[p][1][1]]
  c = main[triplets[p][2][0],triplets[p][2][1]]
  plt.imshow(a, cmap = 'gray')
  plt.show()
  plt.imshow(b, cmap = 'gray')
  plt.show()
  plt.imshow(c, cmap = 'gray')
  plt.show()
  
  print("---------------------------")

FP number  275 :


TypeError: ignored

In [0]:
train_data=[]
for i in range(1024):
  train_data.append([main[triplets[i,0,0],triplets[i,0,1]],main[triplets[i,1,0],triplets[i,1,1]],main[triplets[i,2,0],triplets[i,2,1]]])
train_data=np.array(train_data)
print(train_data.shape)

(1024, 3, 112, 92)


In [0]:
def base_net(ishape):
  s = Sequential(name = 'core_network')
  nfil = [4, 8, 16]
  ksize = 3
  
  s.add(Conv2D(filters = 64, kernel_size = (7,7),strides=2, input_shape = ishape, padding = 'valid', activation = 'relu', data_format = 'channels_first', name = 'Conv1'))
  s.add(MaxPooling2D(pool_size = (3,3),strides=2, data_format = 'channels_first', name = 'Pool1'))
  s.add(keras.layers.BatchNormalization(axis=1,momentum=0.99,epsilon=0.001,name=rnorm1))


  s.add(Conv2D(filters = 64, kernel_size = (1,1),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv2a'))
  s.add(Conv2D(filters = 192, kernel_size = (3,3),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv2'))
  s.add(keras.layers.BatchNormalization(axis=1,momentum=0.99,epsilon=0.001,name=rnorm2))
  s.add(MaxPooling2D(pool_size = (3,3),strides=2, data_format = 'channels_first', name = 'Pool2'))

  s.add(Conv2D(filters = 192, kernel_size = (1,1),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv3a'))
  s.add(Conv2D(filters =384 , kernel_size = (3,3),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv3'))
  s.add(MaxPooling2D(pool_size = (3,3),strides=2, data_format = 'channels_first', name = 'Pool3'))
  
  s.add(Conv2D(filters = 384, kernel_size = (1,1),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv4a'))
  s.add(Conv2D(filters = 256, kernel_size = (3,3),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv4'))
  s.add(Conv2D(filters = 256, kernel_size = (1,1),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv5a'))
  s.add(Conv2D(filters = 256, kernel_size = (3,3), strides=1,input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv5'))
  s.add(Conv2D(filters = 256, kernel_size = (1,1),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv6a'))
  s.add(Conv2D(filters = 256, kernel_size = (3,3),strides=1, input_shape = ishape, padding = 'same', activation = 'relu', data_format = 'channels_first', name = 'Conv6'))
  s.add(MaxPooling2D(pool_size = (3,3),strides=2,data_format = 'channels_first', name = 'Pool4'))
  tf.concat([s],axis, name='concat')
  
  

  
  s.add(Dense((1,32,128), activation = 'relu', name = 'fc1'))

  s.add(Dense((1,32,128), activation = 'relu', name = 'fc2'))
  
  s.add(Dense((1,1,128), activation = 'relu', name = 'Dense50'))
  
  return s